Olá! Este Notebook é um desafio de projeto Explorando IA Generativa em um Pipeline de ETL com Python, do Santander Bootcamp 2023 - Ciência de Dados com Python.

Estarei usando dados de um projeto de análise de negócios que trabalhei no passado. No caso, o negócio era uma importadora de vinhos e nos foi enviado dados de venda dos últimos 5 anos no formato excel.

A ideia é elaborar uma pipeline ETL simples usando a API da OpenAI para gerar conteúdo. E o meu projeto consiste em gerar informação de harmonização para cada um dos rótulos no portifólio.

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('relatorio_2016_2022.xlsx')

Abaixo, das colunas existentes no relatório, iremos necessitar apenas de três:
'COD PRODUTO', 'DESCRIÇÃO' e 'REGIAO'

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51382 entries, 0 to 51381
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DATA           51382 non-null  datetime64[ns]
 1   CLIENTE        51382 non-null  object        
 2   VENDEDOR       51382 non-null  object        
 3   Nome Fantasia  40895 non-null  object        
 4   Bairro         51382 non-null  object        
 5   Cidade         51382 non-null  object        
 6   UF             51382 non-null  object        
 7   NF             51382 non-null  int64         
 8   COD PRODUTO    51382 non-null  object        
 9   DESCRIÇÃO      51382 non-null  object        
 10  PREÇO          51382 non-null  float64       
 11  QTD            51382 non-null  int64         
 12  TOT            51382 non-null  float64       
 13  FORNECEDOR     51382 non-null  object        
 14  REGIAO         51382 non-null  object        
dtypes: datetime64[ns](1

In [4]:
df[['COD PRODUTO', 'DESCRIÇÃO', 'REGIAO']].head()

,COD PRODUTO,DESCRIÇÃO,REGIAO
0,46727,VINHO TINTO PORTAL CARMENERE RESERVA,CHILE - VALE CENTRAL
1,46729,VINHO BRANCO PORTAL SAUVIGNON BLANC RESERVA,CHILE - VALE CENTRAL
2,L12280,VINHO TINTO PORTAL CABERNET/CARMENERE,CHILE - VALE CENTRAL
3,L892686,VINHO TINTO LOMA LARGA CABERNET FRANC,CHILE - CASABLANCA
4,46726,VINHO TINTO PORTAL CABERNET SAUVIGNON RESERVA,CHILE - VALE CENTRAL


Também só iremos gerar conteúdo para os produtos que foram vendidos no ano de 2022.

In [5]:
wines = df[df.DATA > "2021-12-31"][['COD PRODUTO', 'DESCRIÇÃO', 'REGIAO']].groupby('DESCRIÇÃO').first().reset_index()

In [6]:
wines.head()

,DESCRIÇÃO,COD PRODUTO,REGIAO
0,VINHO BRANCO AFRICAN HERITAGE CHARDONNAY,AHCHA,AFRICA DO SUL - WESTERN CAPE
1,VINHO BRANCO AFRICAN HERITAGE CHENIN BLANC,AHCHEB,AFRICA DO SUL - COASTAL REGION
2,VINHO BRANCO AFRICAN HERITAGE SAUVIGNON BLANC,AHSB,AFRICA DO SUL - WESTERN CAPE
3,VINHO BRANCO AGUIA MOURA RESERVA,AMOURABR,PORTUGAL - DOURO
4,VINHO BRANCO ESPUMANTE - NUÀ BRUT,CDNUAB,ITALIA - PIEMONTE


Com os dados devidamente extraídos iremos agora adicionar conteúdo a cada rótulo do portifólio, usando a API.

In [7]:
import openai

f = open('openAI_token.txt', 'r')
token = f.readline()

openai.api_key = token

Abaixo, criei esta função que retorna dicas de harmonização baseado no título do produto.

In [8]:
def generate_harmonization_text(wine):
    completion = openai.ChatCompletion.create(
        model = "gpt-4",
        messages = [
            {
                "role" : "system",
                "content" : "Você é um sommelier."
            },
            {
                "role" : "user",
                "content": f"Crie dicas de harmonização para o vinho {wine} com até 256 caracteres."
            }
        ]
    )
    
    return completion.choices[0].message.content.strip('\"')

Adicionando a coluna de HARMONIZACAO ao DataFrame dos produtos.

In [9]:
wines["HARMONIZACAO"] = wines.REGIAO.apply(lambda x: "")

Abaixo será feita a iteração do DataFrame, adicionando as informações de harmonização à coluna correspondente.

In [ ]:
for i, row in wines.iterrows():
    harmoniz = generate_harmonization_text(row["DESCRIÇÃO"])
    wines.at[i, "HARMONIZACAO"] = harmoniz

Uma prévia do resultado.

In [ ]:
wines.HARMONIZACAO.head()

Por fim, exporto o resultado em um novo arquivo excel.

In [ ]:
wines.to_excel('wines_harmonized.xlsx')